# Control Flow 

### Compound Expression

Sometimes it is convenient to have a single expression which evaluates several subexpressions in order, returning the value of the last subexpression as its value. There are two Julia constructs that accomplish this: begin blocks and ; chains. The value of both compound expression constructs is that of the last subexpression. Here's an example of a begin block:

In [2]:
z = begin
           x = 1
           y = 2
           x + y
       end

3

Since these are fairly small, simple expressions, they could easily be placed onto a single line, which is where the ; chain syntax comes in handy:

In [3]:
z = (x = 1; y = 2; x + y)

3

This syntax is particularly useful with the terse single-line function definition form introduced in Functions. Although it is typical, there is no requirement that begin blocks be multiline or that ; chains be single-line:

In [5]:
begin x = 1; y = 2; x + y end

3

In [6]:
(x = 1;
 y = 2;
 x + y)

3

---
### Conditional Evaluation

Conditional evaluation allows portions of code to be evaluated or not evaluated depending on the value of a boolean expression. Here is the anatomy of the if-elseif-else conditional syntax:

In [7]:
if x < y
    println("x is less than y")
elseif x > y
    println("x is greater than y")
else
    println("x is equal to y")
end

x is less than y


If the condition expression x < y is true, then the corresponding block is evaluated; otherwise the condition expression x > y is evaluated, and if it is true, the corresponding block is evaluated; if neither expression is true, the else block is evaluated. Here it is in action:

In [8]:
function test(x,y)
    if x < y 
        println("x is less than y")
    elseif x>y
        println("x is greater than y")
    else
        println("x is equal to y")
    end 
end 

test (generic function with 1 method)

In [9]:
test(1,2)

x is less than y


In [10]:
test(2,2)

x is equal to y


In [11]:
test(1,1)

x is equal to y


The elseif and else blocks are optional, and as many elseif blocks as desired can be used. The condition expressions in the if-elseif-else construct are evaluated until the first one evaluates to true, after which the associated block is evaluated, and no further condition expressions or blocks are evaluated.

if blocks are "leaky", i.e. they do not introduce a local scope. This means that new variables defined inside the if clauses can be used after the if block, even if they weren't defined before. So, we could have defined the test function above as


In [12]:
function test(x,y)
    if x<y
        relation = "less than"
    elseif x==y
        relation = "equal to"
    else 
        relation = "greater than"
    end 
    println("x is ", relation, " y.")
end 

test (generic function with 1 method)

In [15]:
test(1,1)

x isequal toy.


In [16]:
test(4,5)

x isless thany.


In [17]:
test(5,4)

x isgreater thany.


The variable relation is declared inside the if block, but used outside. However, when depending on this behavior, make sure all possible code paths define a value for the variable. The following change to the above function results in a runtime error.

In [18]:
function test(x,y)
    if x<y 
        relation = "less than"
    elseif x==y
        relation = "equal to"
    end 
    println("x is ", relation, "y.")
end 

test (generic function with 1 method)

In [19]:
test(1,2)

x is less thany.


In [21]:
test(2,2)

x is equal toy.


In [22]:
test(2,1)

LoadError: UndefVarError: relation not defined

In [23]:
x = 3

if x>0
    "positive!"
else 
    "negative..."
end 

"positive!"

Unlike C, MATLAB, Perl, Python, and Ruby – but like Java, and a few other stricter, typed languages – it is an error if the value of a conditional expression is anything but true or false:

In [25]:
if 1
    println("true")
end 

LoadError: TypeError: non-boolean (Int64) used in boolean context

This error indicates that the conditional was of the wrong type: Int64 rather than the required Bool.

In [27]:
if true 
    println("true")
end 

true


The expression a, before the ?, is a condition expression, and the ternary operation evaluates the expression b, before the :, if the condition a is true or the expression c, after the :, if it is false. Note that the spaces around ? and : are mandatory: an expression like a?b:c is not a valid ternary expression (but a newline is acceptable after both the ? and the :).

The easiest way to understand this behavior is to see an example. In the previous example, the println call is shared by all three branches: the only real choice is which literal string to print. This could be written more concisely using the ternary operator. For the sake of clarity, let's try a two-way version first:

In [28]:
x = 1; y = 2;

In [29]:
println(x < y ? "less than" : "not less than")

less than


In [30]:
x = 1 ; y = 0 ;

In [31]:
println(x<y ? "less than" : "not less than")

not less than


If the expression x < y is true, the entire ternary operator expression evaluates to the string "less than" and otherwise it evaluates to the string "not less than". The original three-way example requires chaining multiple uses of the ternary operator together:

In [33]:
test(x, y) = println(x < y ? "x is less than y"    :
                            x > y ? "x is greater than y" : "x is equal to y")

test (generic function with 1 method)

In [34]:
test(1,2)

x is less than y


In [35]:
test(2,1)

x is greater than y


In [36]:
test(1,1)

x is equal to y


To facilitate chaining, the operator associates from right to left.

It is significant that like if-elseif-else, the expressions before and after the : are only evaluated if the condition expression evaluates to true or false, respectively:

In [37]:
v(x) = (println(x); x) 

v (generic function with 1 method)

In [38]:
1 < 2 ? v("yes") : v("no")

yes


"yes"

In [39]:
 1 > 2 ? v("yes") : v("no")

no


"no"

---
### Short-Circuit Evaluation 

The && and || operators in Julia correspond to logical “and” and “or” operations, respectively, and are typically used for this purpose. However, they have an additional property of short-circuit evaluation: they don't necessarily evaluate their second argument, as explained below. (There are also bitwise & and | operators that can be used as logical “and” and “or” without short-circuit behavior, but beware that & and | have higher precedence than && and || for evaluation order.)

Short-circuit evaluation is quite similar to conditional evaluation. The behavior is found in most imperative programming languages having the && and || boolean operators: in a series of boolean expressions connected by these operators, only the minimum number of expressions are evaluated as are necessary to determine the final boolean value of the entire chain. Some languages (like Python) refer to them as and (&&) and or (||). Explicitly, this means that:

In the expression a && b, the subexpression b is only evaluated if a evaluates to true.
In the expression a || b, the subexpression b is only evaluated if a evaluates to false.
The reasoning is that a && b must be false if a is false, regardless of the value of b, and likewise, the value of a || b must be true if a is true, regardless of the value of b. Both && and || associate to the right, but && has higher precedence than || does. It's easy to experiment with this behavior:

In [40]:
t(x) = (println(x); true)

t (generic function with 1 method)

In [41]:
f(x) = (println(x); false) 

f (generic function with 1 method)

In [42]:
t(1) && t(2)

1
2


true

In [43]:
t(1) && f(2)

1
2


false

In [44]:
f(1) && t(2)

1


false

In [45]:
f(1) && f(2)

1


false

In [46]:
t(1) || t(2)

1


true

In [47]:
t(1) || f(2)

1


true

In [48]:
f(1) || t(2)

1
2


true

In [49]:
f(1) || f(2)

1
2


false

In [50]:
function fact(n::Int)
    n >=0 || error("n must be non-negative")
    n == 0 && return 1
    n * fact(n-1)
end

fact (generic function with 1 method)

In [51]:
fact(5)

120

In [52]:
fact(0)

1

In [53]:
fact(10)

3628800

In [54]:
fact(-1)

LoadError: n must be non-negative

In [55]:
f(1) & t(2)

1
2


false

In [56]:
t(1) | t(2)

1
2


true

Just like condition expressions used in if, elseif or the ternary operator, the operands of && or || must be boolean values (true or false). Using a non-boolean value anywhere except for the last entry in a conditional chain is an error:

In [57]:
1 && true 

LoadError: TypeError: non-boolean (Int64) used in boolean context

On the other hand, any type of expression can be used at the end of a conditional chain. It will be evaluated and returned depending on the preceding conditionals:

In [58]:
true && (x=(1,2,3))

(1, 2, 3)

In [59]:
false && (x= (1,2,3))

false

---
### Repeated Evaluation: Loops 

There are two constructs for repeated evaluation of expressions: the while loop and the for loop. Here is an example of a while loop:

In [62]:
# While Loop 
i=1;
while i <= 5 
    println(i)
    global i+=1 
end 

1
2
3
4
5


In [63]:
# for Loop 
for i = 1:5
    println(i)
end

1
2
3
4
5


In [65]:
for j=2:15
    println(j)
end

2
3
4
5
6
7
8
9
10
11
12
13
14
15


the for loop construct can iterate over any container. In these cases, the alternative (but fully equivalent) keyword in or ∈ is typically used instead of =, since it makes the code read more clearly:

In [66]:
for i in [1,4,0]
           println(i)
       end

1
4
0


In [67]:
for s ∈ ["foo","bar","baz"]
           println(s)
       end

foo
bar
baz


It is sometimes convenient to terminate the repetition of a while before the test condition is falsified or stop iterating in a for loop before the end of the iterable object is reached. This can be accomplished with the break keyword:

In [68]:
i =1; 
while true
   println(i)
   if i >= 5
       break
   end
   global i += 1
end

1
2
3
4
5


In [69]:
for j = 1:1000
   println(j)
   if j >= 5
       break
   end
end

1
2
3
4
5


Without the break keyword, the above while loop would never terminate on its own, and the for loop would iterate up to 1000. These loops are both exited early by using break.

In other circumstances, it is handy to be able to stop an iteration and move on to the next one immediately. The continue keyword accomplishes this:

In [70]:
for i = 1:10
   if i % 3 != 0
       continue
   end
   println(i)
end

3
6
9


This is a somewhat contrived example since we could produce the same behavior more clearly by negating the condition and placing the println call inside the if block. In realistic usage there is more code to be evaluated after the continue, and often there are multiple points from which one calls continue.

Multiple nested for loops can be combined into a single outer loop, forming the cartesian product of its iterables:

In [71]:
for i = 1:2, j = 3:4
    println((i,j))
end

(1, 3)
(1, 4)
(2, 3)
(2, 4)


Multiple containers can be iterated over at the same time in a single for loop using zip:

In [72]:
for (j, k) in zip([1 2 3], [4 5 6 7])
   println((j,k))
end

(1, 4)
(2, 5)
(3, 6)


Using zip will create an iterator that is a tuple containing the subiterators for the containers passed to it. The zip iterator will iterate over all subiterators in order, choosing the iith element of each subiterator in the iith iteration of the for loop. Once any of the subiterators run out, the for loop will stop.

---
## Exception handling